In [74]:
# SET CONFIG AND PARAMETERS
path = '/home/xilinx/jupyter_notebooks/qick/TII/characterizationScripts/zcu216/configuration/'
with open(path+"setConfigV1.py", 'r') as f:
    code1 = f.read()
    exec(code1)
with open(path+"setParametersSpinq10q69.py", 'r') as f:
    code2 = f.read()
    exec(code2) 

Configuration and parameters set


In [75]:
class QubitSpectroscopyProgram(NDAveragerProgram):
    
    def ns2cycles(self, ns, gen_ch=None, ro_ch=None):
        return self.us2cycles(ns/1000, gen_ch=gen_ch, ro_ch=ro_ch)

    def initialize(self):
        cfg=self.cfg
        sweeps = []
        probe_ch    = cfg["probe_ch"]        
        feedback_ch = cfg["feedback_ch"]        
        ro_pulse_shape  = cfg["ro_pulse_shape"]
        ro_pulse_length = self.ns2cycles(cfg["ro_pulse_length"], gen_ch=probe_ch)  
        self.declare_gen(ch=probe_ch, 
                         nqz=cfg["probe_ch_nqz"], 
                         mux_freqs=self.cfg['ro_pulse_frequency'], 
                         mux_gains=self.cfg['ro_pulse_gain'])
        self.set_pulse_registers(ch=probe_ch, 
                                 style=ro_pulse_shape, 
                                 length=ro_pulse_length, 
                                 mask=[0,1,2,3])        
        for iCh, ch in enumerate(feedback_ch):
            self.declare_readout(ch=ch, 
                                 freq=self.cfg['acquisition_frequency'][iCh], 
                                 length = self.ns2cycles(self.cfg['acquisition_length'], ro_ch=ch), 
                                 gen_ch=probe_ch)  
            
        f_expts = int(2 * cfg["qubit_freq_width"]/cfg["qubit_freq_step"])+1
        
        
        for index, qubit in enumerate(cfg["qubits"]):
            drive_ch = cfg["drive_ch"][index]
            drive_pulse_DAC_frequency = self.freq2reg(cfg[qubit]["drive_pulse_frequency"], gen_ch=drive_ch)
            drive_pulse_DAC_phase     = self.deg2reg(cfg[qubit]["drive_pulse_relative_phase"], gen_ch=drive_ch)
            drive_pulse_DAC_gain      = int(cfg[qubit]["drive_pulse_gain"] * MAX_GAIN) #int(cfg["q1_drive_pulse_gain"] * MAX_GAIN)
            drive_pulse_shape         = cfg[qubit]["drive_pulse_shape"]
            drive_pulse_length        = self.ns2cycles(cfg[qubit]["drive_pulse_length"], gen_ch=drive_ch) #self.ns2cycles(cfg["q_length"], gen_ch=q1_drive_ch)
            # Sweeper parameters
            f_start= cfg[qubit]["drive_pulse_frequency"] - cfg["qubit_freq_width"]
            f_stop = cfg[qubit]["drive_pulse_frequency"] + cfg["qubit_freq_width"]             
 
            self.declare_gen(ch=drive_ch, 
                             nqz=cfg["drive_ch_nqz"])
            self.add_gauss(ch=drive_ch, 
                           name="drive_pulse", 
                           sigma=int(cfg[qubit]["sigma"] * 384), 
                           length=drive_pulse_length,)
            self.set_pulse_registers(ch=drive_ch, 
                                     gain=drive_pulse_DAC_gain, 
                                     freq=self.freq2reg(f_start, gen_ch=drive_ch), 
                                     phase=drive_pulse_DAC_phase, 
                                     #style="const",
                                     #length= q1_drive_pulse_length
                                     style=drive_pulse_shape, 
                                     waveform="drive_pulse"
                                    )      
        
            # ---------- sweep defination starts from here -----------------
            # get freq registers of the generator channel (check 04_Reading_Math_Writing for more details about QickRegister object)
            self.res_r_freq = self.get_gen_reg(drive_ch, "freq")
            # add pulse freq sweep, first added will be first swept
            sweeps.append(QickSweep(self, self.res_r_freq,  f_start, f_stop, f_expts))
            
        self.add_sweep(merge_sweeps(sweeps))
        
        
        self.synci(500)  # give processor some time to configure pulses
    
    def body(self):
        cfg=self.cfg
        for index, qubit in enumerate(cfg["qubits"]):
            drive_ch = cfg["drive_ch"][index]
            drive_pulse_start = self.ns2cycles(cfg[qubit]["drive_pulse_start"], gen_ch=drive_ch)
            self.pulse(ch=cfg["drive_ch"][index], t=drive_pulse_start)
           

        ro_pulse_start       = self.ns2cycles(cfg[cfg["qubits"][0]]["drive_pulse_length"] + cfg["lag"], gen_ch=cfg["probe_ch"])
        #ro_pulse_start       = self.ns2cycles( cfg["lag"], gen_ch=cfg["probe_ch"])

        self.measure(pulse_ch= cfg["probe_ch"], 
                     adcs = cfg["feedback_ch"], 
                     adc_trig_offset=self.ns2cycles(cfg["delay_before_acquisition"]), 
                     wait=True, 
                     syncdelay=self.ns2cycles(cfg["relaxation_time"]), 
                     t=ro_pulse_start
                    )
        

In [76]:
def run_qubit_spectroscopy():
    experiment_points = int(2*experiment_cfg["qubit_freq_width"]/experiment_cfg["qubit_freq_step"])+1
#     q1_freq = drive_cfg["q1_drive_pulse_frequency"]
#     q2_freq = drive_cfg["q2_drive_pulse_frequency"]
#     q3_freq = drive_cfg["q3_drive_pulse_frequency"]
#     q4_freq = drive_cfg["q4_drive_pulse_frequency"]
    results_1 = np.zeros(experiment_points)
    results_2 = np.zeros(experiment_points)
    results_3 = np.zeros(experiment_points)
    results_4 = np.zeros(experiment_points)

    set_sweetspots()
    #frequency_range = np.linspace(-30, 10, experiment_points)
    #results=[]
    program_cfg = {**channel_cfg,**readout_cfg,**qubit_cfg, **experiment_cfg} 
    program = QubitSpectroscopyProgram(soccfg=soc_cfg, cfg=program_cfg)
    freq_range, avg_di, avg_dq = program.acquire(soc, progress=True)
    #print( "range:", np.transpose(np.array(freq_range)).shape, "I:",np.transpose(np.array(avg_di[1])).shape, "Q",np.array(avg_dq[2]).shape)

    #print( "range:", freq_range[0], "I:",avg_di, "Q",avg_dq)
    #print(results_matrix1[1].shape, results_matrix2[2].shape)
    results_1 = np.squeeze(np.abs(avg_di[0] + 1j*avg_dq[0]))
    results_2 = np.squeeze(np.abs(avg_di[1] + 1j*avg_dq[1]))
    results_3 = np.squeeze(np.abs(avg_di[2] + 1j*avg_dq[2]))
    results_4 = np.squeeze(np.abs(avg_di[3] + 1j*avg_dq[3]))
        
    reset_bias()
    return freq_range, results_1,results_2,results_3,results_4

In [77]:
experiment_cfg = {  "qubit_freq_width": 50,      # 
                    "qubit_freq_step": 0.5}        # 

In [78]:
freq_range, r1,r2,r3,r4 = run_qubit_spectroscopy()

  0%|          | 0/402000 [00:00<?, ?it/s]

In [79]:

# Create a 2x2 subplot figure
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=(channel_cfg["qubits"][0], channel_cfg["qubits"][1], channel_cfg["qubits"][2], channel_cfg["qubits"][3]),
    x_title="Frequency (MHz)",
    y_title="Amplitude"
)

# Add each qubit's result to a different subplot
fig.add_trace(go.Scatter(x=np.array(freq_range[0])[:,0], y=r1, mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.array(freq_range[0])[:,1], y=r2, mode='lines'), row=1, col=2)
fig.add_trace(go.Scatter(x=np.array(freq_range[0])[:,2], y=r3, mode='lines'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.array(freq_range[0])[:,3], y=r4, mode='lines'), row=2, col=2)

# Update layout with a main title
fig.update_layout(
    title={'text': "Qubit Spectroscopy",
            'font': {
            'size': 24  # Set the font size
        },
          },
    height = 800, width = 1000,
    title_x= 0.5,    
    showlegend=False
)

# Show the plot
fig.show()